# Plotting Results
This notebook contains the code used to generate plots for the technical report, including feature importance results and visualizations of model performance. 
Note that the plots are not displayed to reduce the size of the notebook. You can uncomment the plot lines below or view them in the `img` directory.

In [1]:
import pandas as pd
import joblib
import altair as alt
import glob
import sys
import os
import numpy as np
from pathlib import Path

sys.path.append(os.path.abspath(".."))
alt.data_transformers.enable("vegafusion")
from src.models.feat_selection import ImportanceFeatureSelector

/Users/benjaminfrizzell/miniforge3/envs/mds-afforest-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Feature Importance

### Permutation Importance

In [2]:
perm_plots = []
model_order=['Logistic Regression','Random Forest','Gradient Boosting']
for threshold in [50,60,70,80]:   
    gbm_perm = joblib.load(f'../models/{threshold}/fitted_gradient_boosting_permute.joblib')
    rf_perm = joblib.load(f'../models/{threshold}/fitted_random_forest_permute.joblib')
    lr_perm = joblib.load(f'../models/{threshold}/fitted_logistic_regression_permute.joblib')

    permutation_df = pd.DataFrame({
        'mean':np.concatenate((
            gbm_perm.values['importances_mean'],
            rf_perm.values['importances_mean'],
            lr_perm.values['importances_mean'])),
        'std':np.concatenate((
            gbm_perm.values['importances_std'],
            rf_perm.values['importances_std'],
            lr_perm.values['importances_std']
            ))/np.sqrt(5),
        'model':['Gradient Boosting']*15 + ['Random Forest']*15 + ['Logistic Regression']*15,
        'feature':gbm_perm.plot_data.index.to_list()*3
    })
    permutation_df['ci_lower'] = permutation_df['mean'] - 1.96*permutation_df['std']
    permutation_df['ci_upper'] = permutation_df['mean'] + 1.96*permutation_df['std']

    feat_order = gbm_perm.plot_data.index.to_list()

    bars = alt.Chart(permutation_df).mark_bar().encode(
        x=alt.X('feature:N',sort=feat_order, title='',axis=alt.Axis(labelAngle=45)),
        xOffset=alt.XOffset('model:N',sort=model_order),
        y=alt.Y('mean:Q', title='',scale=alt.Scale(domain=[None,0.25])),
        color=alt.Color('model:N',title='Model',sort=model_order),
    )

    error_bars = alt.Chart(permutation_df).mark_rule().encode(
        x=alt.X('feature:N',sort=feat_order),
        xOffset=alt.XOffset('model:N',sort=model_order),
        y='ci_lower:Q',
        y2='ci_upper:Q',
        color=alt.value('black')
    )

    cap_top = alt.Chart(permutation_df).mark_tick(
        color='black',
        thickness=1,
        width=6
    ).encode(
        x=alt.X('feature:N', sort=feat_order),
        xOffset=alt.XOffset('model:N',sort=model_order),
        y='ci_upper:Q'
    )

    # Bottom caps
    cap_bottom = alt.Chart(permutation_df).mark_tick(
        color='black',
        thickness=1,
        width=6
    ).encode(
        x=alt.X('feature:N', sort=feat_order),
        xOffset=alt.XOffset('model:N',sort=model_order),
        y='ci_lower:Q'
    )


    permutation_plot = (bars + error_bars + cap_bottom + cap_top).properties(
        title=f'{threshold}%',
        width=450
        )
    perm_plots.append(permutation_plot)
    
title = alt.Chart({'values':[{}]}).mark_text(
    text='Permutation Feature Importance',
    fontSize=20,
    font='Helvetica',
    dy=-10
).encode().properties(
    width=225,
    height=2
)

# Remove x-axis tick labels but keep ticks and gridlines
perm_plots[0] = perm_plots[0].encode(
    x=alt.X('feature:N', axis=alt.Axis(labels=False, title=None))
)
perm_plots[1] = perm_plots[1].encode(
    x=alt.X('feature:N', axis=alt.Axis(labels=False, title=None))
)

# Remove y-axis tick labels but keep ticks and gridlines
perm_plots[1] = perm_plots[1].encode(
    y=alt.Y('mean:Q', axis=alt.Axis(labels=False, title=None))
)
perm_plots[3] = perm_plots[3].encode(
    y=alt.Y('mean:Q', axis=alt.Axis(labels=False, title=None))
)

perm_importance_plot = alt.vconcat(title, (perm_plots[0] & perm_plots[2]) | (perm_plots[1] & perm_plots[3])).configure_view(stroke=None)
#perm_importance_plot

In [3]:
perm_importance_plot.save('../img/feature_importance_permutation.png',ppi=500)

### SHAP Importance

In [4]:
shap_plots = []
model_order=['Logistic Regression','Random Forest','Gradient Boosting']
for threshold in [50,60,70,80]:   
    gbm_shap = joblib.load(f'../models/{threshold}/fitted_gradient_boosting_shap.joblib')
    rf_shap = joblib.load(f'../models/{threshold}/fitted_random_forest_shap.joblib')
    lr_shap = joblib.load(f'../models/{threshold}/fitted_logistic_regression_shap.joblib')

    shap_df = pd.DataFrame({
        'mean':np.concatenate((
            gbm_shap.plot_data,
            rf_shap.plot_data,
            lr_shap.plot_data)),
        'model':['Gradient Boosting']*15 + ['Random Forest']*15 + ['Logistic Regression']*15,
        'feature':gbm_shap.plot_data.index.to_list()*3
    })

    feat_order = gbm_shap.plot_data.index.to_list()

    bars = alt.Chart(shap_df).mark_bar().encode(
        x=alt.X('feature:N',sort=feat_order, title='',axis=alt.Axis(labelAngle=45)),
        xOffset=alt.XOffset('model:N',sort=model_order),
        y=alt.Y('mean:Q', title=''),
        color=alt.Color('model:N',title='Model',sort=model_order),
    ).properties(
        title=f'{threshold}%',
        width=450
        )
    shap_plots.append(bars)


# Remove x-axis tick labels but keep ticks and gridlines
shap_plots[0] = shap_plots[0].encode(
    x=alt.X('feature:N', axis=alt.Axis(labels=False, title=None))
)
shap_plots[1] = shap_plots[1].encode(
    x=alt.X('feature:N', axis=alt.Axis(labels=False, title=None))
)

# Remove y-axis tick labels but keep ticks and gridlines
shap_plots[1] = shap_plots[1].encode(
    y=alt.Y('mean:Q', axis=alt.Axis(labels=False, title=None))
)
shap_plots[3] = shap_plots[3].encode(
    y=alt.Y('mean:Q', axis=alt.Axis(labels=False, title=None))
)

title = alt.Chart({'values':[{}]}).mark_text(
    text='SHAP Feature Importance',
    fontSize=20,
    font='Helvetica',
    dy=-10
).encode().properties(
    width=235,
    height=2
)


shap_importance_plot = alt.vconcat(title,(shap_plots[0] & shap_plots[2]) | (shap_plots[1] & shap_plots[3]))
#shap_importance_plot

In [5]:
shap_importance_plot.save('../img/feature_importance_shap.png',ppi=500)

### RFE Cross-Validation Rankings

In [6]:
rfe_plots = []
for threshold in [50,60,70,80]:
    # get models   
    gbm_rfe = joblib.load(f'../models/{threshold}/fitted_gradient_boosting_rfecv.joblib')
    rf_rfe = joblib.load(f'../models/{threshold}/fitted_random_forest_rfecv.joblib')
    lr_rfe = joblib.load(f'../models/{threshold}/fitted_logistic_regression_rfecv.joblib')
    
    # construct dataframe for plotting
    rfecv_df = pd.DataFrame({
        'feature': 
            [name.split('__')[1] for name in gbm_rfe.named_steps['columntransformer'].get_feature_names_out()] + 
            [name.split('__')[1] for name in rf_rfe.named_steps['columntransformer'].get_feature_names_out()] + 
            [name.split('__')[1] for name in lr_rfe.named_steps['columntransformer'].get_feature_names_out()],
        'model': ['Gradient Boosting']*15 + ['Random Forest']*15 + ['Logistic Regression']*15,
        'ranking': np.concatenate((
            gbm_rfe.named_steps['rfecv'].ranking_,
            rf_rfe.named_steps['rfecv'].ranking_,
            lr_rfe.named_steps['rfecv'].ranking_
            ))
    })
    
    rfe_heatmap = alt.Chart(rfecv_df).mark_rect().encode(
    x=alt.X('feature:N',title='',axis=alt.Axis(labelAngle=45)),
    y=alt.Y('model:N',sort=model_order,title=''),
    color=alt.Color('ranking:O',title='Importance Ranking',scale=alt.Scale(scheme='greens', reverse=True))
    ).properties(
        title=f'{threshold}%',
        width=300
        )
    rfe_plots.append(rfe_heatmap)        



# Remove x-axis tick labels but keep ticks and gridlines
rfe_plots[0] = rfe_plots[0].encode(
    x=alt.X('feature:N', axis=alt.Axis(labels=False, title=None))
)
rfe_plots[1] = rfe_plots[1].encode(
    x=alt.X('feature:N', axis=alt.Axis(labels=False, title=None))
)

# Remove y-axis tick labels but keep ticks and gridlines
rfe_plots[1] = rfe_plots[1].encode(
    y=alt.Y('model:N', axis=alt.Axis(labels=False, title=None))
)
rfe_plots[3] = rfe_plots[3].encode(
    y=alt.Y('model:N', axis=alt.Axis(labels=False, title=None))
)

title = alt.Chart({'values':[{}]}).mark_text(
    text='RFE Importance',
    fontSize=20,
    font='Helvetica',
    dy=-10
).encode().properties(
    width=145,
    height=2
)

rfe_importance_plot = alt.vconcat(title,(rfe_plots[0] & rfe_plots[2]) | (rfe_plots[1] & rfe_plots[3]))
#rfe_importance_plot

In [7]:
rfe_importance_plot.save('../img/feature_importance_rfe.png',ppi=500)

## Model Performance

### Precision Recall Curves

In [8]:
precision_recall_dfs = []

# gather precision recall plots for all models
for model_name in ['gradient_boosting','random_forest','logistic_regression']:
    for file in Path('..').glob(f"results/*/{model_name}_pr_curve.csv"):
        df = pd.read_csv(file)
        df['Threshold'] = file.parent.name + '%'
        df['Model'] = model_name.replace('_', ' ').title()
        precision_recall_dfs.append(df)

precision_recall_df = pd.concat(precision_recall_dfs, ignore_index=True)

In [9]:
precision_recall_plot = alt.Chart(precision_recall_df).mark_line().encode(
    x=alt.X('Recall:Q'),
    y=alt.Y('Precision:Q'),
    color=alt.Color('Threshold:O',sort=['50%','60%','70%','80%'],scale=alt.Scale(scheme='greens'))
).facet(
    column=alt.Column('Model:N', title='Precision-Recall Curves')
).configure_view(
    strokeWidth=0  # Removes the border around each facet
).configure_header(
    titleFontSize=20,  # Size of the facet title
)
#precision_recall_plot

In [10]:
precision_recall_plot.save('../img/evaluation_pr-curve.png',ppi=500)

### ROC Curves

In [11]:
roc_dfs = []

# gather precision recall plots for all models
for model_name in ['gradient_boosting','random_forest','logistic_regression']:
    for file in Path('..').glob(f"results/*/{model_name}_roc_curve.csv"):
        df = pd.read_csv(file)
        df['Threshold'] = file.parent.name + '%'
        df['Model'] = model_name.replace('_', ' ').title()
        roc_dfs.append(df)

roc_df = pd.concat(roc_dfs, ignore_index=True)

alt.data_transformers.enable("vegafusion")
roc_plot = alt.Chart(roc_df).mark_line().encode(
    x=alt.X('False Positive Rate:Q'),
    y=alt.Y('True Positive Rate:Q'),
    color=alt.Color('Threshold:O',sort=['50%','60%','70%','80%'],scale=alt.Scale(scheme='greens'))
).facet(
    column=alt.Column('Model:N', title='ROC Curves')
).configure_view(
    strokeWidth=0  # Removes the border around each facet
).configure_header(
    titleFontSize=20,  # Size of the facet title
)
#roc_plot

In [12]:
roc_plot.save('../img/evaluation_roc-curve.png',ppi=500)

### Confusion Matrix

In [61]:
conf_matrix_dfs = []

for model_name in ['gradient_boosting','random_forest','logistic_regression']:
    for file in Path('..').glob(f"results/*/{model_name}_confusion_matrix.csv"):
        conf_matrix = pd.read_csv(file,index_col=0)
        conf_matrix = conf_matrix.reset_index().melt(id_vars='index')
        conf_matrix.columns = ['True', 'Predicted', 'Count']
        conf_matrix['Threshold'] = file.parent.name + '%'
        conf_matrix['Model'] = model_name.replace('_', ' ').title()
        conf_matrix_dfs.append(conf_matrix)

conf_matrix_df = pd.concat(conf_matrix_dfs,ignore_index=True)

In [123]:
heatmap = alt.Chart(conf_matrix_df).mark_rect().encode(
    x=alt.X('Predicted:N', title='',axis=alt.Axis(labelAngle=0)),
    y=alt.Y('True:N', title=''),
    color=alt.Color('Count:Q', scale=alt.Scale(scheme='greens'),legend=None),
).properties(
    width=200,
    height=200
)

text = alt.Chart(conf_matrix_df).mark_text(baseline='middle').encode(
    x='Predicted:N',
    y='True:N',
    text='Count:Q'
).properties(
    width=200,
    height=200
)

conf_mat_plot = (heatmap+text).facet(
    row=alt.Row('Model:N',title='',header=alt.Header(labelAngle=0,labelPadding=7,labelOrient='right')),
    column=alt.Column('Threshold:O',sort=['50%','60%','70%','80%'],title='Confusion Matrices')
).configure_view(
    strokeWidth=0  # Removes the border around each facet
).configure_header(
    titleFontSize=20,  # Size of the facet title
)

#conf_mat_plot

In [124]:
conf_mat_plot.save('../img/evaluation_conf_matrix.png',ppi=500)

## Other

### Target Distribution Plot

In [18]:
train_lookup = pd.read_parquet('../data/processed/train_lookup.parquet')

In [19]:
target_barplot = alt.Chart(train_lookup).mark_bar().encode(
    x = alt.X('target:Q').bin(maxbins=40).title('Survival Rate (%)'),
    y = alt.Y('count()').title('Count'),
    color = alt.Color('Age:N').title('Site Age (yrs)'),
    
).properties(
    title='Survival Rate Distribution (Training set)'
)
target_barplot

alt.Chart(...)

In [20]:
target_barplot.save('../img/target_barplot.png',ppi=500)